In [9]:
import tensorflow as tf
from tensorflow import keras
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [10]:
dataset_path = '/home/ridwan/Documents/aiub/cvpr/pic/'

In [11]:
# Load dataset with resizing images to the same dimensions
def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = os.listdir(dataset_path)
    for index, name in enumerate(tqdm(label_names)):
        folder_path = os.path.join(dataset_path, name)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            if image is not None:  # Check if the image was successfully loaded
                image = cv2.resize(image, (224, 224))  # Resize images to 224x224 pixels
                images.append(image)
                labels.append(index)
            else:
                print(f"Warning: Could not load image {image_path}")
    images = np.array(images, dtype='float32') / 255.0  # Normalize images
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=len(label_names))
    return images, labels, label_names

images, labels, label_names = load_dataset(dataset_path)


100%|██████████| 7/7 [00:03<00:00,  2.27it/s]


In [12]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [13]:
# Build CNN model
def build_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

model = build_model(num_classes=len(label_names))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


2024-03-01 22:12:57.521859: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [14]:
# Train model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/20
3/3 [==============================] - 5s 1s/step - loss: 5.2126 - accuracy: 0.1389 - val_loss: 2.6651 - val_accuracy: 0.1111
Epoch 2/20
3/3 [==============================] - 4s 1s/step - loss: 2.1019 - accuracy: 0.2222 - val_loss: 2.0909 - val_accuracy: 0.5000
Epoch 3/20
3/3 [==============================] - 4s 1s/step - loss: 2.4423 - accuracy: 0.3194 - val_loss: 2.0629 - val_accuracy: 0.5000
Epoch 4/20
3/3 [==============================] - 4s 1s/step - loss: 2.0357 - accuracy: 0.4306 - val_loss: 2.0966 - val_accuracy: 0.2222
Epoch 5/20
3/3 [==============================] - 4s 1s/step - loss: 1.6968 - accuracy: 0.5278 - val_loss: 1.7846 - val_accuracy: 0.1667
Epoch 6/20
3/3 [==============================] - 4s 1s/step - loss: 1.3229 - accuracy: 0.6250 - val_loss: 1.4942 - val_accuracy: 0.5556
Epoch 7/20
3/3 [==============================] - 4s 1s/step - loss: 1.0595 - accuracy: 0.7083 - val_loss: 1.4201 - val_accuracy: 0.6111
Epoch 8/20
3/3 [=========================

In [ ]:
import cv2
import numpy as np

def real_time_recognition(model, label_names):
    # Initialize the webcam (use 0 as the parameter to select the default webcam)
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    while True:
        ret, frame = cap.read()  # Read a frame from the webcam
        if not ret:
            break
        
        # Preprocess the frame for prediction (resize, expand dimensions, normalize)
        face = cv2.resize(frame, (224, 224))  # Assuming your model expects 224x224 inputs
        face = np.expand_dims(face, axis=0)  # Add batch dimension
        face = face / 255.0  # Normalize pixel values to [0, 1] as during training
        
        prediction = model.predict(face)  # Make prediction
        predicted_class = label_names[np.argmax(prediction)]  # Get the name of the predicted class
        
        # Display the predicted class on the frame
        cv2.putText(frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Show the frame
        cv2.imshow('Real-time Face Recognition', frame)
        
        # Break the loop when the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Assuming 'model' is your trained model and 'label_names' is a list of class names
# real_time_recognition(model, label_names)
